In [9]:
from functools import partial
import pickle
pickle.load = partial(pickle.load, encoding="latin1")
pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")

# import spoof_detection
import numpy as np
import pandas as pd

import os
import io
import PIL
import dlib
import torch
import requests
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from scipy.misc import imresize, imread
detector = dlib.get_frontal_face_detector()

import os
import time
import torch

import os
import random
import numpy as np
from torch.utils.data import Dataset
from skimage.color import rgb2hsv, hsv2rgb
from scipy.misc import imresize
from torch.utils.data.sampler import WeightedRandomSampler
import torch.optim as optim
from sklearn.metrics import log_loss, roc_auc_score
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [122]:
print(torch.__version__)

0.4.0a0+dc4984e


In [3]:
# SIZE = 128
# PATCH_SIZE = 32
# RESOLUTION = 32

# model_dir = '/media/dataserver/datasets/internal/face_spoofing/januari_revolut/spoof_detection/utils'
# MODEL_LOCATION = os.path.join(model_dir, 'network_patchnormlr3')
# MODEL_LOCATION = '/media/dataserver/workspace/blanca/MSDNet-GCN/results/savedir/test/save_models/model_best.pth.tar'

# print(MODEL_LOCATION)

# # load model and set to evaluation mode
# model = torch.load(MODEL_LOCATION).cuda()
# model.eval()
# model.train(False)

In [4]:
# def detect_face(img):
#     ''' Detect faces in the image and return a crop of the largest face. '''
#     # detect faces
#     dets, conf, _ = detector.run(img)
#     # filter detections with conf <= 0
#     dets = [d for d, c in zip(dets,conf) if c > 0]
#     # if at least one face was detected
#     if len(dets) > 0:
#         # calculate face sizes per detection
#         size = [max(d.bottom(), 0) - max(d.top(), 0) + max(d.right(), 0) - max(d.left(), 0) for d in dets]
#         # select detection corresponding with largest face
#         det = dets[np.argmax(size)]
#         # get face positions
#         top, bot, left, right = max(det.top(), 0), max(det.bottom(), 0), max(det.left(), 0), max(det.right(), 0)
#         # return cropped image
#         return img[top:bot, left:right]
#     return None

# def create_heatmap(face_crop):
#     ''' Select uniformly sampled patches from the face_crop and run them through the network. '''
#     face_crop = face_crop.transpose(2, 0, 1)
#     c, h, w = face_crop.shape
#     # create a batch with uniformly sampled patches
#     hr = float(h - PATCH_SIZE) / (RESOLUTION - 1)
#     wr = float(w - PATCH_SIZE) / (RESOLUTION - 1)
#     batch = np.zeros((RESOLUTION**2, 3, PATCH_SIZE, PATCH_SIZE))
#     for y in range(RESOLUTION):
#         for x in range(RESOLUTION):
#             patch = face_crop[:, int(y * hr):int(y * hr + PATCH_SIZE), int(x * wr):int(x * wr + PATCH_SIZE)]
#             patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-8)
#             batch[y * RESOLUTION + x, :, :, :] = patch
#     # network makes a prediction per patch
#     inputs = Variable(torch.from_numpy(batch).float(), volatile=True).cuda()
#     outputs = F.softmax(model(inputs), dim=1).data.cpu().numpy()
#     #score = np.mean(outputs, 0)[0]
#     #outputs = model(inputs).data.cpu().numpy()
#     #score = np.mean(outputs, 0)
#     #score = np.exp(score)[0] / sum(np.exp(score)) 
#     return score

# def predict(image):
#     ''' Turns input image into a spoof attempt prediction. '''
#     # 1. download image
#     #url = 'https://imago.onfido.com/api/{}/{}/download'.format('live_photos', str(photo_id))
#     #im_response = requests.get(url)
#     #try:
#     #    image = imread(io.BytesIO(im_response.content))
#     #except:
#     #    return 'failed: reading image'
#     # filter greyscale images
#     if len(image.shape) is not 3 or image.shape[2] < 3:
#         return 'failed: image shape'
#     # remove possible alpha channel
#     if image.shape[2] > 3: 
#         image = image[:,:,:3]
#     # get exit and rotate
#     #img = PIL.Image.open(io.BytesIO(im_response.content))
#     #if hasattr(img, '_getexif') and img._getexif() is not None and 274 in img._getexif():
#     #    orientation = img._getexif()[274]
#     #    if orientation in [3,4]:
#     #        image = np.rot90(image, 2)
#     #    if orientation in [5,6]:
#     #        image = np.rot90(image, 3)
#     #    if orientation in [7,8]:
#     #        image = np.rot90(image, 1)
#     # detect face in image
#     face_crop = detect_face(image)
#     if face_crop is None:
#         return 'failed: face detection'
#     # resize image
#     face_crop = imresize(face_crop, (SIZE, SIZE))
#     # spoof prediction
#     heatmap = create_heatmap(face_crop)
#     return heatmap
#     # transform heatmap into a spoof prediction (1.0 is spoof attack)
#     spoof_score = heatmap
#     # also return the most likely class
#     #most_likely_class = ['genuine sample', 'picture attack', 'screen attack', 'document attack'][np.argmax(np.mean(heatmap, axis=(0, 1)))]
#     return spoof_score

In [5]:
# load unique warning and passed live_photo_ids
data_dir = '/blanca/datasets/internal/face_spoofing/januari_revolut'
warning_ids = np.load(os.path.join(data_dir, 'warning/warning_ids.npy'))
passed_ids = np.load(os.path.join(data_dir, 'passed/passed_ids.npy'))
all_ids = list(set(np.concatenate((warning_ids, passed_ids))))
all_ids.sort()
print(len(all_ids))
print (len(set(warning_ids)), len(set(passed_ids)))
print (len(set(warning_ids)) + len(set(passed_ids)))
print (len(set(np.concatenate((warning_ids, passed_ids)))))

5000
521 4479
5000
5000


In [6]:
# create table
table = pd.DataFrame(np.zeros((5000,4)), columns=['live_photo_id', 'manual_label', 'spoof_label', 'spoof_score'])
table['live_photo_id'] = all_ids
table['manual_label'] = table['manual_label'].apply(str)
table['spoof_label'] = table['spoof_label'].apply(str)
table['spoof_label'] = 'None'
table['jonne_label'] = ''
# loop through table and set manual_label to warning or passed
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    if photo_id in warning_ids: table.set_value(index, 'manual_label', 'warning')
    if photo_id in passed_ids: table.set_value(index, 'manual_label', 'passed')

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


In [127]:
import torch.nn as nn
import math
import torch
import torch.nn.functional as F
from collections import OrderedDict


'''
from:
https://github.com/kazuto1011/cifar10-pytorch
with an added BatchNorm layer at the start
'''

class BasicBlock(nn.Module):

    def __init__(self, n_in, n_out, stride=1):
        super(BasicBlock, self).__init__()
        self.connection = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(n_in, n_out, 3, stride, 1, bias=False)),
            ('norm1', nn.BatchNorm2d(n_out)),
            ('relu1', nn.ReLU(inplace=True)),
            ('conv2', nn.Conv2d(n_out, n_out, 3, 1, 1, bias=False)),
            ('norm2', nn.BatchNorm2d(n_out)),
        ]))
        self.relu = nn.ReLU(inplace=True)
        self.downsample = nn.Sequential(
            nn.Conv2d(n_in, n_out, 1, stride, bias=False),
            nn.BatchNorm2d(n_out),
        )
        self.stride = stride

    def forward(self, x):
        mapping = self.connection(x)
        if self.stride != 1:
            x = self.downsample(x)
        return self.relu(mapping + x)


class ResidualBlock(nn.Module):

    def __init__(self, n_in, n_out, n_block, stride=1):
        super(ResidualBlock, self).__init__()
        self.blocks = nn.Sequential()
        self.blocks.add_module('block0', BasicBlock(n_in, n_out, stride))
        for i in range(n_block - 1):
            self.blocks.add_module('block{}'.format(i + 1), BasicBlock(n_out, n_out))

    def forward(self, x):
        return self.blocks(x)


class ResNetCifar10(nn.Module):

    def __init__(self, n_classes=2, n_block=3):
        super(ResNetCifar10, self).__init__()
        ch = [16, 32, 64]
        print('HERE HERE HERE')
        self.bn0 = nn.BatchNorm2d(3, affine=True)
        self.features = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, ch[0], 3, 1, 1, bias=False)),
            ('norm1', nn.BatchNorm2d(ch[0])),
            ('relu1', nn.ReLU(inplace=True)),
            ('resb1', ResidualBlock(ch[0], ch[0], n_block)),
            ('resb2', ResidualBlock(ch[0], ch[1], n_block, 2)),
            ('resb3', ResidualBlock(ch[1], ch[2], n_block, 2)),
            ('avgpl', nn.AvgPool2d(8)),
        ]))
        self.fc = nn.Linear(ch[2], n_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        print('HERE HERE HERE')
        x = self.bn0(x)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
SPOOF_MODEL_PATH = 'network_patchnormlr3_state_dict'

# OPTION 0
# spoof_model = torch.load(SPOOF_MODEL_PATH)
# layer = spoof_model._modules['bn0']

# # OPTION 1
# spoof_model = ResNetCifar10()
# spoof_model.load_state_dict(torch.load(SPOOF_MODEL_PATH))

# # OPTION 2
network = ResNetCifar10()
network.load_state_dict(torch.load(SPOOF_MODEL_PATH))

# pretrained_dict = torch.load(SPOOF_MODEL_PATH)                
# model_dict = network.state_dict()

# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}                    
# network.load_state_dict(pretrained_dict)
# print('Pretrained network %s has excessive layers; Only loading layers that are used' % network_label)
#                 except:
#                     print('Pretrained network %s has fewer layers; The following are not initialized:' % network_label)
#                     from sets import Set
#                     not_initialized = Set()
#                     for k, v in pretrained_dict.items():                      
#                         if v.size() == model_dict[k].size():
#                             model_dict[k] = v

#                     for k, v in model_dict.items():
#                         if k not in pretrained_dict or v.size() != pretrained_dict[k].size():
#                             not_initialized.add(k.split('.')[0])                            
#                     print(sorted(not_initialized))
#                     network.load_state_dict(model_dict) 

# network.state_dict()
# pretrained_dict.state_dict().items()



HERE HERE HERE


In [112]:
import os
import io
import PIL
import dlib
import torch
import requests
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from scipy.misc import imresize, imread
detector = dlib.get_frontal_face_detector()

os.environ["CUDA_VISIBLE_DEVICES"] = '7'
# from .test_internal import heatmap, test_network
# from utils.networks import *
# import spoof_detection

SIZE = 256

# MODEL_LOCATION = os.path.join('utils', 'network_patchnormlr3')
# MODEL_LOCATION = os.path.join('utils', 'network')
# load model and set to evaluation mode
# model = torch.load(MODEL_LOCATION).cuda()
# model.eval()
# model.train(False)

def detect_face(img):
    ''' Detect faces in the image and return a crop of the largest face. '''
    # detect faces
    dets, conf, _ = detector.run(img)
    # filter detections with conf <= 0
    dets = [d for d, c in zip(dets,conf) if c > 0]
    # if at least one face was detected
    if len(dets) > 0:
        # calculate face sizes per detection
        size = [max(d.bottom(), 0) - max(d.top(), 0) + max(d.right(), 0) - max(d.left(), 0) for d in dets]
        # select detection corresponding with largest face
        det = dets[np.argmax(size)]
        # get face positions
        top, bot, left, right = max(det.top(), 0), max(det.bottom(), 0), max(det.left(), 0), max(det.right(), 0)
        # return cropped image
        return img[top:bot, left:right]
    return None

def create_heatmap(model, face_crop):
    ''' Select uniformly sampled patches from the face_crop and run them through the network. '''
    face_crop = face_crop.transpose(2, 0, 1)
    c, h, w = face_crop.shape
    # create a batch with uniformly sampled patches
    hr = float(h - PATCH_SIZE) / (RESOLUTION - 1)
    wr = float(w - PATCH_SIZE) / (RESOLUTION - 1)
    batch = np.zeros((RESOLUTION**2, 3, PATCH_SIZE, PATCH_SIZE))
    for y in range(RESOLUTION):
        for x in range(RESOLUTION):
            patch = face_crop[:, int(y * hr):int(y * hr + PATCH_SIZE), int(x * wr):int(x * wr + PATCH_SIZE)]
            #patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-8)
            batch[y * RESOLUTION + x, :, :, :] = patch
    # network makes a prediction per patch
    inputs = Variable(torch.from_numpy(batch).float(), volatile=True).cuda()
    outputs = model(inputs).data.cpu().numpy()
    score = np.mean(outputs, 0)
    score = np.exp(score)[0] / sum(np.exp(score)) 
    return score

def predict(model, photo_id):
    ''' Turns input image into a spoof attempt prediction. '''
    # 1. download image
    url = 'https://imago.onfido.com/api/{}/{}/download'.format('live_photos', str(photo_id))
    im_response = requests.get(url)
    try:
        image = imread(io.BytesIO(im_response.content))
    except:
        return 'failed: reading image'
    # filter greyscale images
    if len(image.shape) is not 3 or image.shape[2] < 3:
        return 'failed: image shape'
    # remove possible alpha channel
    if image.shape[2] > 3: 
        image = image[:,:,:3]
    # get exit and rotate
    img = PIL.Image.open(io.BytesIO(im_response.content))
    if hasattr(img, '_getexif') and img._getexif() is not None and 274 in img._getexif():
        orientation = img._getexif()[274]
        if orientation in [3,4]:
            image = np.rot90(image, 2)
        if orientation in [5,6]:
            image = np.rot90(image, 3)
        if orientation in [7,8]:
            image = np.rot90(image, 1)
    # detect face in image
    face_crop = detect_face(image)
    if face_crop is None:
        return 'failed: face detection'
    # resize image
    face_crop = imresize(face_crop, (SIZE, SIZE))
    h, w, c = face_crop.shape
    # take middle patch and normalize
    dh = (h - 224) / 2
    dw = (w - 224) / 2
    #face_crop = face_crop[dh:-dh, dw:-dw, :] / 127.5 - 1.0
    face_crop = (face_crop - np.mean(face_crop)) / (np.std(face_crop) + 1e-8)
    #face_crop = face_crop / 255.0
    # prepare for network
    face_crop = np.expand_dims(face_crop, 0)
    face_crop = face_crop.transpose(0, 3, 1, 2)
    # run through network
    inputs = Variable(torch.from_numpy(face_crop).float(), volatile=True).cuda()
    
    outputs = model(inputs).data.cpu().numpy()[0]
    outputs = np.exp(outputs[0]) / sum(np.exp(outputs))
    return outputs

In [ ]:
state_dict = torch.load(args.evaluate_from)['state_dict']
model.load_state_dict(state_dict)

In [120]:
msdnet_model_chp = 'checkpoint_117.pth.tar'
# msdnet_model.load_state_dict(torch.load(msdnet_model_chp))
# msdnet_model = torch.load(msdnet_model_chp)

# for i in msdnet_model.items(): print(i)

# for i, j in msdnet_model['state_dict'].items():
#     print(i)
# for i in msdnet_model.keys(): print(i)
# msdnet_model.eval()

epoch
model
state_dict
best_prec1
optimizer


AttributeError: 'dict' object has no attribute 'eval'

In [133]:
photo_id = row['live_photo_id']
photo_id

186370

In [115]:
# load model and set to evaluation mode
model = torch.load(msdnet_model_chp)['state_dict'].cuda()
# model = torch.load(MODEL_LOCATION).cuda()
model.eval()
model.train(False)

# loop through table and set spoof_detection to the predicted spoof score using spoof_detection
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    prediction = predict(model, photo_id)
    
    if type(prediction) is str:
        table.set_value(index, 'spoof_label', prediction)
    else:
        table.set_value(index, 'spoof_score', prediction)
        if prediction < 0.925:
            table.set_value(index, 'spoof_label', 'spoof')
        else:
            table.set_value(index, 'spoof_label', 'passed')
        
    print (index)

AttributeError: 'collections.OrderedDict' object has no attribute 'cuda'

#### ResNetCifar10

In [134]:
import torch.nn as nn
import math
import torch
import torch.nn.functional as F
from collections import OrderedDict

'''
from:
https://github.com/kazuto1011/cifar10-pytorch
with an added BatchNorm layer at the start
'''

class BasicBlock(nn.Module):

    def __init__(self, n_in, n_out, stride=1):
        super(BasicBlock, self).__init__()
        self.connection = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(n_in, n_out, 3, stride, 1, bias=False)),
            ('norm1', nn.BatchNorm2d(n_out)),
            ('relu1', nn.ReLU(inplace=True)),
            ('conv2', nn.Conv2d(n_out, n_out, 3, 1, 1, bias=False)),
            ('norm2', nn.BatchNorm2d(n_out)),
        ]))
        self.relu = nn.ReLU(inplace=True)
        self.downsample = nn.Sequential(
            nn.Conv2d(n_in, n_out, 1, stride, bias=False),
            nn.BatchNorm2d(n_out),
        )
        self.stride = stride

    def forward(self, x):
        mapping = self.connection(x)
        if self.stride != 1:
            x = self.downsample(x)
        return self.relu(mapping + x)


class ResidualBlock(nn.Module):

    def __init__(self, n_in, n_out, n_block, stride=1):
        super(ResidualBlock, self).__init__()
        self.blocks = nn.Sequential()
        self.blocks.add_module('block0', BasicBlock(n_in, n_out, stride))
        for i in range(n_block - 1):
            self.blocks.add_module('block{}'.format(i + 1), BasicBlock(n_out, n_out))

    def forward(self, x):
        return self.blocks(x)


class ResNetCifar10(nn.Module):

    def __init__(self, n_classes=2, n_block=3):
        super(ResNetCifar10, self).__init__()
        ch = [16, 32, 64]
        print('HERE HERE HERE')
        self.bn0 = nn.BatchNorm2d(3, affine=True)
        self.features = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, ch[0], 3, 1, 1, bias=False)),
            ('norm1', nn.BatchNorm2d(ch[0])),
            ('relu1', nn.ReLU(inplace=True)),
            ('resb1', ResidualBlock(ch[0], ch[0], n_block)),
            ('resb2', ResidualBlock(ch[0], ch[1], n_block, 2)),
            ('resb3', ResidualBlock(ch[1], ch[2], n_block, 2)),
            ('avgpl', nn.AvgPool2d(8)),
        ]))
        self.fc = nn.Linear(ch[2], n_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        print('HERE HERE HERE')
        x = self.bn0(x)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
SPOOF_MODEL_PATH = 'network_patchnormlr3_state_dict'

network = ResNetCifar10()
network.load_state_dict(torch.load(SPOOF_MODEL_PATH))

model = network.cuda()
model.eval()
model.train(False)

# loop through table and set spoof_detection to the predicted spoof score using spoof_detection
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    prediction = predict(model, photo_id)
    
    if type(prediction) is str:
        table.set_value(index, 'spoof_label', prediction)
    else:
        table.set_value(index, 'spoof_score', prediction)
        if prediction < 0.925:
            table.set_value(index, 'spoof_label', 'spoof')
        else:
            table.set_value(index, 'spoof_label', 'passed')
        
    print (index)

HERE HERE HERE


/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:71: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


HERE HERE HERE


/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:95: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:107: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


RuntimeError: size mismatch, m1: [1 x 4096], m2: [64 x 2] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:249

In [ ]:
def detect_face(img):
    ''' Detect faces in the image and return a crop of the largest face. '''
    # detect faces
    dets, conf, _ = detector.run(img)
    # filter detections with conf <= 0
    dets = [d for d, c in zip(dets,conf) if c > 0]
    # if at least one face was detected
    if len(dets) > 0:
        # calculate face sizes per detection
        size = [max(d.bottom(), 0) - max(d.top(), 0) + max(d.right(), 0) - max(d.left(), 0) for d in dets]
        # select detection corresponding with largest face
        det = dets[np.argmax(size)]
        # get face positions
        top, bot, left, right = max(det.top(), 0), max(det.bottom(), 0), max(det.left(), 0), max(det.right(), 0)
        # return cropped image
        return img[top:bot, left:right]
    return None

def predict(model, photo_id):
    ''' Turns input image into a spoof attempt prediction. '''
    # 1. download image
    url = 'https://imago.onfido.com/api/{}/{}/download'.format('live_photos', str(photo_id))
    im_response = requests.get(url)
    try:
        image = imread(io.BytesIO(im_response.content))
    except:
        return 'failed: reading image'
    # filter greyscale images
    if len(image.shape) is not 3 or image.shape[2] < 3:
        return 'failed: image shape'
    # remove possible alpha channel
    if image.shape[2] > 3: 
        image = image[:,:,:3]
    # get exit and rotate
    img = PIL.Image.open(io.BytesIO(im_response.content))
    if hasattr(img, '_getexif') and img._getexif() is not None and 274 in img._getexif():
        orientation = img._getexif()[274]
        if orientation in [3,4]:
            image = np.rot90(image, 2)
        if orientation in [5,6]:
            image = np.rot90(image, 3)
        if orientation in [7,8]:
            image = np.rot90(image, 1)
    # detect face in image
    face_crop = detect_face(image)
    if face_crop is None:
        return 'failed: face detection'
    # resize image
    face_crop = imresize(face_crop, (SIZE, SIZE))
    h, w, c = face_crop.shape
    # take middle patch and normalize
    dh = (h - 224) / 2
    dw = (w - 224) / 2
    #face_crop = face_crop[dh:-dh, dw:-dw, :] / 127.5 - 1.0
    face_crop = (face_crop - np.mean(face_crop)) / (np.std(face_crop) + 1e-8)
    #face_crop = face_crop / 255.0
    # prepare for network
    face_crop = np.expand_dims(face_crop, 0)
    face_crop = face_crop.transpose(0, 3, 1, 2)
    # run through network
    inputs = Variable(torch.from_numpy(face_crop).float(), volatile=True).cuda()

    return inputs

In [ ]:
photo_id = row['live_photo_id']
prediction = predict(model, photo_id)
    
outputs = model(inputs).data.cpu().numpy()[0]
outputs = np.exp(outputs[0]) / sum(np.exp(outputs))

In [137]:
for index, row in table.iterrows():
    print(index, row)

0 live_photo_id    186370
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 0, dtype: object
1 live_photo_id     313856
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 1, dtype: object
2 live_photo_id    439505
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 2, dtype: object
3 live_photo_id     549714
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 3, dtype: object
4 live_photo_id     558681
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 4, dtype: object
5 live_photo_id     578999
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 5, dtype: object
6 live_photo_id     606128
manual_label     warning
spoof_label         None
spoof_score            0
jonne_la

Name: 187, dtype: object
188 live_photo_id     663388
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 188, dtype: object
189 live_photo_id     663404
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 189, dtype: object
190 live_photo_id     667139
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 190, dtype: object
191 live_photo_id     667581
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 191, dtype: object
192 live_photo_id     667692
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 192, dtype: object
193 live_photo_id     667819
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 193, dtype: object
194 live_photo_id     668367
manual_label     warni

409 live_photo_id     759705
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 409, dtype: object
410 live_photo_id     759815
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 410, dtype: object
411 live_photo_id     759820
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 411, dtype: object
412 live_photo_id     760308
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 412, dtype: object
413 live_photo_id     760490
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 413, dtype: object
414 live_photo_id     760503
manual_label     warning
spoof_label         None
spoof_score            0
jonne_label             
Name: 414, dtype: object
415 live_photo_id     760543
manual_label     warning
spoof_label         No

Name: 605, dtype: object
606 live_photo_id    809118
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 606, dtype: object
607 live_photo_id    809120
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 607, dtype: object
608 live_photo_id    809122
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 608, dtype: object
609 live_photo_id    809124
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 609, dtype: object
610 live_photo_id    809126
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 610, dtype: object
611 live_photo_id    809127
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 611, dtype: object
612 live_photo_id    809130
manual_label     passed
spoof_label        None
spoof

Name: 783, dtype: object
784 live_photo_id    809579
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 784, dtype: object
785 live_photo_id    809580
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 785, dtype: object
786 live_photo_id    809581
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 786, dtype: object
787 live_photo_id    809582
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 787, dtype: object
788 live_photo_id    809590
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 788, dtype: object
789 live_photo_id    809592
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 789, dtype: object
790 live_photo_id    809595
manual_label     passed
spoof_label        None
spoof

...

Name: 3910, dtype: object
3911 live_photo_id    817304
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 3911, dtype: object
3912 live_photo_id    817305
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 3912, dtype: object
3913 live_photo_id    817308
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 3913, dtype: object
3914 live_photo_id    817309
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 3914, dtype: object
3915 live_photo_id    817311
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 3915, dtype: object
3916 live_photo_id    817312
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 3916, dtype: object
3917 live_photo_id    817313
manual_label     passed
spoof_label    

Name: 4169, dtype: object
4170 live_photo_id    817868
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4170, dtype: object
4171 live_photo_id    817869
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4171, dtype: object
4172 live_photo_id    817872
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4172, dtype: object
4173 live_photo_id    817874
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4173, dtype: object
4174 live_photo_id    817875
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4174, dtype: object
4175 live_photo_id    817878
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4175, dtype: object
4176 live_photo_id    817879
manual_label     passed
spoof_label    

Name: 4379, dtype: object
4380 live_photo_id    818384
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4380, dtype: object
4381 live_photo_id    818387
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4381, dtype: object
4382 live_photo_id    818388
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4382, dtype: object
4383 live_photo_id    818389
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4383, dtype: object
4384 live_photo_id    818391
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4384, dtype: object
4385 live_photo_id    818392
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4385, dtype: object
4386 live_photo_id    818397
manual_label     passed
spoof_label    

Name: 4608, dtype: object
4609 live_photo_id    819047
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4609, dtype: object
4610 live_photo_id    819052
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4610, dtype: object
4611 live_photo_id    819054
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4611, dtype: object
4612 live_photo_id    819055
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4612, dtype: object
4613 live_photo_id    819057
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4613, dtype: object
4614 live_photo_id    819065
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4614, dtype: object
4615 live_photo_id    819066
manual_label     passed
spoof_label    

Name: 4875, dtype: object
4876 live_photo_id    819992
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4876, dtype: object
4877 live_photo_id    819994
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4877, dtype: object
4878 live_photo_id    820001
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4878, dtype: object
4879 live_photo_id    820003
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4879, dtype: object
4880 live_photo_id    820005
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4880, dtype: object
4881 live_photo_id    820032
manual_label     passed
spoof_label        None
spoof_score           0
jonne_label            
Name: 4881, dtype: object
4882 live_photo_id    820035
manual_label     passed
spoof_label    

In [7]:
# get labeled spoofs
import os
folder = '../warning/sorted'
folders = ['gen', 'pop', 'pod', 'pos', 'etc', 'unknown']
labels = ['genuine selfie', 'picture attack', 'document attack', 'screen attack', 'camera defect', 'uncertain spoof']
def check_folder(live_id):
    for sub, label in zip(folders, labels):
        if os.path.exists(os.path.join(folder,sub,str(live_id)+'.png')):
            return label

In [8]:
# loop through table
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    label = check_folder(photo_id)
    table.set_value(index, 'jonne_label', label)
    

In [9]:
print table

      live_photo_id manual_label             spoof_label  spoof_score  \
0            186370       passed                   spoof     0.996201   
1            313856      warning                  passed     0.002434   
2            439505       passed                   spoof     0.998563   
3            549714      warning                  passed     0.062495   
4            558681      warning                  passed     0.002560   
5            578999      warning                  passed     0.000165   
6            606128      warning                  passed     0.011200   
7            606469      warning                  passed     0.000093   
8            606763      warning                   spoof     0.941019   
9            607571      warning                  passed     0.007935   
10           607616      warning                  passed     0.004856   
11           607829      warning                  passed     0.076798   
12           608191      warning                  p

In [10]:
writer = pd.ExcelWriter('anti-spoof-patch-results-patchnormlr3-averagesoftmax.xlsx')
table.to_excel(writer,'Sheet1')
writer.save()